# STA 243: Homework 1

>Hongye Li
<br>
>Ming Zhao
<br>
>05/06/2020

## 1
see pdf file

## 2

***
#### (a)

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
train = pd.read_csv("train.data.csv")
test = pd.read_csv("test.data.csv")

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15129 entries, 0 to 15128
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     15129 non-null  int64  
 1   id             15129 non-null  int64  
 2   date           15129 non-null  object 
 3   price          15129 non-null  int64  
 4   bedrooms       15129 non-null  int64  
 5   bathrooms      15129 non-null  float64
 6   sqft_living    15129 non-null  int64  
 7   sqft_lot       15129 non-null  int64  
 8   floors         15129 non-null  float64
 9   waterfront     15129 non-null  int64  
 10  view           15129 non-null  int64  
 11  condition      15129 non-null  int64  
 12  grade          15129 non-null  int64  
 13  sqft_above     15129 non-null  int64  
 14  sqft_basement  15129 non-null  int64  
 15  yr_built       15129 non-null  int64  
 16  yr_renovated   15129 non-null  int64  
 17  zipcode        15129 non-null  int64  
 18  lat   

In [4]:
y_train = train["price"]
y_test = test["price"]

X_train = train[["bedrooms", "bathrooms", "sqft_living", "sqft_lot"]]
X_test = test[["bedrooms", "bathrooms", "sqft_living", "sqft_lot"]]

In [5]:
# train model
lr = LinearRegression() 
lr.fit(X_train,y_train)

LinearRegression()

In [6]:
# obtain coefficients w/o interaction
lr_beta = np.matrix(np.append(lr.intercept_,lr.coef_)).T
lr_beta

matrix([[ 8.08329891e+04],
        [-5.92969566e+04],
        [ 3.68165620e+03],
        [ 3.16685731e+02],
        [-4.26736772e-01]])

In [7]:
# calculate R2 on training data
y_pred_train = lr.predict(X_train)
r2_train = r2_score(y_train, y_pred_train)
print(r2_train)

0.5101138530794578


In [8]:
# calculate R2 on testing data
y_pred_test = lr.predict(X_test)
r2_test = r2_score(y_test, y_pred_test)
print(r2_test)

0.5049944614037056


***
#### (b)

In [9]:
fancy = pd.read_csv("fancyhouse.csv")

In [10]:
X_fancy = fancy[["bedrooms", "bathrooms", "sqft_living", "sqft_lot"]]

In [11]:
# predict price of Bill Gates' house
y_pred_fancy = lr.predict(X_fancy)
print(y_pred_fancy)

[15436769.53822435]


***
#### (c)

In [12]:
X_train2 = X_train.copy()
X_test2 = X_test.copy()

X_train2["add_term"] = X_train2["bedrooms"] * X_train2["bathrooms"]
X_test2["add_term"] = X_test2["bedrooms"] * X_test2["bathrooms"]

In [13]:
# train model
lr2 = LinearRegression() 
lr2.fit(X_train2,y_train)

LinearRegression()

In [14]:
# obtain coefficients w interaction
lr_beta2 = np.matrix(np.append(lr2.intercept_,lr2.coef_)).T
lr_beta2

matrix([[ 3.20752748e+05],
        [-1.28706409e+05],
        [-1.11008869e+05],
        [ 3.08724841e+02],
        [-4.24577540e-01],
        [ 3.37536051e+04]])

In [15]:
# calculate R2 on training data
y_pred_train2 = lr2.predict(X_train2)
r2_train2 = r2_score(y_train, y_pred_train2)
print(r2_train2)

0.5173532927738305


In [16]:
# calculate R2 on testing data
y_pred_test2 = lr2.predict(X_test2)
r2_test2 = r2_score(y_test, y_pred_test2)
print(r2_test2)

0.5105355458591379


***
#### (d)

In [17]:
def LRGD (X, y, step_size = 0.001, n_iter = 1000, eps = 1e-6):
    '''
    Gradient Descent for linear regression
    X is a n*p matrix
    y is a n*1 vector
    b is a p*1 vector
    '''
    
    n, p = X.shape
    
    ones = np.ones((n, 1))
    X = np.append(ones, X, axis = 1)
    
    b = np.zeros((p+1, 1))
    
    for t in range(n_iter):
        
        gradient = -2/n * X.T @ (y - X @ b) 
        b = b - step_size * gradient
        gradient_norm = np.linalg.norm(gradient)
        
        if gradient_norm < eps:
            break
            
    return b

In [18]:
def st2unst(stb, y, X):
    '''
    Transformation on standardized coefficients
    '''
    
    p = X.shape[1]
    unstb = np.zeros((p+1, 1))
    unstb_xbar = []
    
    for i in range(p):
        unstb[i+1] = stb[i+1] * np.std(y) / np.std(X[:,i])
        unstb_xbar.append(unstb[i+1] * np.mean(X[:,i])) 
        
    unstb[0] = np.mean(y) - sum(unstb_xbar)
    
    return unstb


In [19]:
def predict(x, betahat):
    """
    linear regression prediction
    """
    
    return betahat[0] + x @ betahat[1:]

In [20]:
# convert dataframe to matrix
X_train = np.matrix(X_train)
y_train = np.matrix(y_train).T
X_train2 = np.matrix(X_train2)

X_test = np.matrix(X_test)
y_test = np.matrix(y_test).T
X_test2 = np.matrix(X_test2)

In [21]:
# standardize data
X_train_scaled = (X_train - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)
y_train_scaled = (y_train - np.mean(y_train)) / np.std(y_train)
X_train2_scaled = (X_train2 - np.mean(X_train2, axis=0)) / np.std(X_train2, axis=0)

X_test_scaled = (X_test - np.mean(X_test, axis=0)) / np.std(X_test, axis=0)
y_test_scaled = (y_test - np.mean(y_test)) / np.std(y_test)
X_test2_scaled = (X_test2 - np.mean(X_test2, axis=0)) / np.std(X_test2, axis=0)

In [22]:
# search for best step_size
step_sizes = [0.1, 0.01, 0.001, 0.0001]
beta = []
beta2 = []
for i in range(len(step_sizes)):   
    beta.append(LRGD(X_train_scaled, y_train_scaled, step_size = step_sizes[i])) # models w/o interaction
    beta2.append(LRGD(X_train2_scaled, y_train_scaled, step_size = step_sizes[i])) # models w interaction

In [23]:
# transform standardized coefficients to originals
coef = []
coef2 = []
for i in range(len(step_sizes)):
    coef.append(st2unst(beta[i], y_train, X_train))
    coef2.append(st2unst(beta2[i], y_train, X_train2))

In [24]:
# determine best step_size by calculating average difference in coefficients between lr and gd methods
diff = []
diff2 = []
for i in range(len(step_sizes)):
    dis = np.mean(np.abs(lr_beta - coef[i]))
    dis2 = np.mean(np.abs(lr_beta2 - coef2[i]))
    diff.append(dis)
    diff2.append(dis2)

In [25]:
print(diff)
print(diff2)

[0.22681480681664698, 654.0081452777072, 53004.42627743493, 65687.07594797036]
[236.0990878360914, 47586.07636608571, 100243.84273616994, 62193.45760884956]


In [26]:
# use 0.1 as final step_size
gd_beta = coef[0]
gd_beta2 = coef2[0]

In [27]:
# compare lr to gd results for models w/o interaction
np.append(lr_beta, gd_beta, 1)

matrix([[ 8.08329891e+04,  8.08325713e+04],
        [-5.92969566e+04, -5.92968521e+04],
        [ 3.68165620e+03,  3.68226741e+03],
        [ 3.16685731e+02,  3.16685128e+02],
        [-4.26736772e-01, -4.26734934e-01]])

In [28]:
# compare lr to gd results for models w interaction
np.append(lr_beta2, gd_beta2, 1)

matrix([[ 3.20752748e+05,  3.20009972e+05],
        [-1.28706409e+05, -1.28494194e+05],
        [-1.11008869e+05, -1.10646983e+05],
        [ 3.08724841e+02,  3.08729394e+02],
        [-4.24577540e-01, -4.24548946e-01],
        [ 3.37536051e+04,  3.36538920e+04]])

In [29]:
# perform part(a)

# calculate R2 on training data
gd_y_pred_train = predict(X_train, gd_beta)
gd_r2_train = r2_score(y_train, gd_y_pred_train)
print(gd_r2_train)

# calculate R2 on testing data
gd_y_pred_test = predict(X_test, gd_beta)
gd_r2_test = r2_score(y_test, gd_y_pred_test)
print(gd_r2_test)

0.5101138530785188
0.5049944887919995


In [30]:
# perform part(b)

# predict price of Bill Gates' house
gd_y_pred_fancy = predict(np.matrix(X_fancy), gd_beta)
print(gd_y_pred_fancy)

[[15436755.47672183]]


In [31]:
# perform part(c)

# calculate R2 on training data
gd_y_pred_train2 = predict(X_train2, gd_beta2)
gd_r2_train2 = r2_score(y_train, gd_y_pred_train2)
print(gd_r2_train2)

# calculate R2 on testing data
gd_y_pred_test2 = predict(X_test2, gd_beta2)
gd_r2_test2 = r2_score(y_test, gd_y_pred_test2)
print(gd_r2_test2)

0.5173532282363673
0.5105385532390928


***
#### (e)

In [32]:
def LRSGD (X, y, step_size = 0.001, n_iter = 100000, eps = 1e-6):
    '''
    Stochastic Gradient Descent for OLS: 
        1. one observarion per iteration
        2. sample without replacement
    X is a n*p matrix
    y is a n*1 vector
    b is a p*1 vector
    '''
    n, p = X.shape
    
    ones = np.ones((n, 1))
    X = np.append(ones, X, axis = 1)
        
    b = np.zeros((p+1, 1))
    
    data = np.append(y, X, axis = 1)   
    
    # calculate the times of running through the full data
    if n_iter % n == 0: 
        I = n_iter // n
    else:
        I = n_iter // n + 1
    
    # obtain the sample index for each iteration
    index = []
    for i in range(I): 
        np.random.seed(i)
        index = np.append(index, np.random.choice(n, n, replace=False))    
    index = index[:n_iter]   
    
    t = 0
    gradient_norm = 1
    
    while t < n_iter and gradient_norm >= eps:
        
        for ind in index: 
        
            data_temp = data[int(ind)]
            mini_y = data_temp[:, 0]
            mini_X = data_temp[:, 1:]
    
            gradient = -2 * mini_X.T @ (mini_y - mini_X @ b)
            b = b - step_size * gradient
            
            t += 1
            
            if t % n == 0:
                gradient_norm = np.linalg.norm(gradient)
                if gradient_norm < eps:
                    break
                    
    return b

In [33]:
def LRSGD_alter (X, y, step_size = 0.001, n_iter = 1000, k = 100, eps = 1e-6):
    '''
    Stochastic Gradient Descent for OLS: 
        1. multiple observarions per iteration
        2. sample with replacement
    X is a n*p matrix
    y is a n*1 vector
    b is a p*1 vector
    '''
    
    n, p = X.shape
    
    ones = np.ones((n, 1))
    X = np.append(ones, X, axis = 1)
        
    b = np.zeros((p+1, 1))
    
    data = np.append(y, X, axis = 1)

    t = 0
    gradient_norm = 1
    
    while t < n_iter and gradient_norm >= eps:
        
        np.random.seed(t)
        index = np.random.choice(n, k, replace=False)
        data_temp = data[index]
        mini_y = data_temp[:, 0]
        mini_X = data_temp[:, 1:]
    
        gradient = -2/k * mini_X.T @ (mini_y - mini_X @ b)
        b = b - step_size * gradient
        
        t += 1
        gradient_norm = np.linalg.norm(gradient)
    
    return b

In [34]:
# search for best step_size
step_sizes = [0.01, 0.001, 0.0001, 0.00001]
beta = []
beta2 = []
for i in range(len(step_sizes)):   
    beta.append(LRSGD(X_train_scaled, y_train_scaled, step_size = step_sizes[i])) # models w/o interaction
    beta2.append(LRSGD(X_train2_scaled, y_train_scaled, step_size = step_sizes[i])) # models w interaction

In [35]:
# transform standardized coefficients to originals
coef = []
coef2 = []
for i in range(len(step_sizes)):
    coef.append(st2unst(beta[i], y_train, X_train))
    coef2.append(st2unst(beta2[i], y_train, X_train2))

In [36]:
# determine best step_size by calculating average difference in coefficients between lr and sgd methods
diff = []
diff2 = []
for i in range(len(step_sizes)):
    dis = np.mean(np.abs(lr_beta - coef[i]))
    dis2 = np.mean(np.abs(lr_beta2 - coef2[i]))
    diff.append(dis)
    diff2.append(dis2)

In [37]:
print(diff)
print(diff2)

[31522.254363902273, 11035.401876874985, 1366.1500260349385, 53686.811228807914]
[65321.32148624168, 11785.1296189597, 47869.92478163912, 101032.85001803543]


In [38]:
# use 0.0001 as final step_size for model w/o interaction
# use 0.001 as final step_size for model w interaction
sgd_beta = coef[2]
sgd_beta2 = coef2[1]

In [39]:
# compare lr to gd results for models w/o interaction
np.append(lr_beta, sgd_beta, 1)

matrix([[ 8.08329891e+04,  7.83401252e+04],
        [-5.92969566e+04, -5.93424198e+04],
        [ 3.68165620e+03,  7.97168242e+03],
        [ 3.16685731e+02,  3.14402618e+02],
        [-4.26736772e-01, -5.40341358e-01]])

In [40]:
# compare lr to gd results for models w interaction
np.append(lr_beta2, sgd_beta2, 1)

matrix([[ 3.20752748e+05,  3.70765690e+05],
        [-1.28706409e+05, -1.41001309e+05],
        [-1.11008869e+05, -1.16985097e+05],
        [ 3.08724841e+02,  3.02261869e+02],
        [-4.24577540e-01, -4.73413535e-01],
        [ 3.37536051e+04,  3.61738015e+04]])

In [41]:
# perform part(a)

# calculate R2 on training data
sgd_y_pred_train = predict(X_train, sgd_beta)
sgd_r2_train = r2_score(y_train, sgd_y_pred_train)
print(sgd_r2_train)

# calculate R2 on testing data
sgd_y_pred_test = predict(X_test, sgd_beta)
sgd_r2_test = r2_score(y_test, sgd_y_pred_test)
print(sgd_r2_test)

0.5099282729266595
0.5044362207387216


In [42]:
# perform part(b)

# predict price of Bill Gates' house
sgd_y_pred_fancy = predict(np.matrix(X_fancy), sgd_beta)
print(sgd_y_pred_fancy)

[[15401446.89744386]]


In [43]:
# perform part(c)

# calculate R2 on training data
sgd_y_pred_train2 = predict(X_train2, sgd_beta2)
sgd_r2_train2 = r2_score(y_train, sgd_y_pred_train2)
print(sgd_r2_train2)

# calculate R2 on testing data
sgd_y_pred_test2 = predict(X_test2, sgd_beta2)
sgd_r2_test2 = r2_score(y_test, sgd_y_pred_test2)
print(sgd_r2_test2)

0.5165499943027383
0.5097266407396426


## 3
see pdf file